In [ ]:
import sys,os,signal
from time import time

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import torch
from torch import Tensor
from torchvision import transforms

from IPython import display

import mnist, models, utils

root = 'data/'

In [ ]:
def worker_init(args):
    signal.signal(signal.SIGINT, signal.SIG_IGN)
    
batch_size = 256
kwargs = {'num_workers': 2, 'pin_memory': True, 'worker_init_fn': worker_init}

train_set = mnist.MNIST(root=root, train=True, one_hot=True, transform=transforms.ToTensor(),download=True)
test_set = mnist.MNIST(root=root, train=False, one_hot=True, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=False, **kwargs)

In [ ]:
seed_size = 128

g = models.Generator(seed_size).cuda()
f = models.Discriminator().cuda()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(14,7))

lr = 3e-4

foptimizer = torch.optim.Adam(f.parameters(), lr=lr, betas=(0,0.9))
goptimizer = torch.optim.Adam(g.parameters(), lr=lr, betas=(0,0.9))

losses = []

i = 0
t0 = time()
scores = []
epochs = 40
for epoch in range(epochs):        
    for x,_ in train_loader:
        x = x.cuda()
        
        #
        # Problem 5: Calculate the gan updates to optimize
        #   (1) The discriminator f, using foptimizer.
        #   (2) The generator g, using goptimizer.
        #
        # Track your generator loss by appending its loss to losses.
        #
        
        losses.append(loss.detach().cpu().numpy())
        
        if i % 100 == 0:
            g.eval()
            y = g(torch.randn(batch_size, seed_size).cuda())
            display.clear_output(wait=True)
            utils.print_images(y.detach().cpu().numpy(), ax[0], d=28)
            ax[1].cla(); ax[1].plot(losses)
            display.display(plt.gcf())
            print(i, time()-t0)
            g.train()
            t0 = time()
        i += 1